#### Location Inference
Add inference about location to EduSense video pipeline. 


In [ ]:
import pymongo
import pprint
from sshtunnel import SSHTunnelForwarder
import base64
from io import BytesIO
import math

import numpy as np
from PIL import Image, ImageDraw
from pandas import DataFrame
from collections import OrderedDict

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

from scipy.spatial import distance as dist
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from sklearn.decomposition import PCA
from shapely.geometry import Point
from shapely.geometry import LineString

import seaborn as sns; sns.set()

In [ ]:
print("Connecting to MongoDB...")

MONGO_HOST = ""
MONGO_DB = ""
MONGO_USER = ""
MONGO_PASS = ""

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

server.start()
# local port connection
client = pymongo.MongoClient('127.0.0.1', server.local_bind_port)
db = client[MONGO_DB]
print("\033[92mConnection successful\033[0m")

print("Opening sample video collection")
collection = db['']
print("\033[92mOpened document \033[0m \n")

#### Instructor movement inference

In [ ]:
def getMovementData(collection, subject = "student", return_errors = False):
    positions_i = []
    frames = []

    for item in collection:
        if item["channel"]=="instructor":
            for person in item["people"]:

                s_gaze = person["inference"]["head"]["gazeVector"]
                if len(s_gaze) == 2:
                    positions_i.append(s_gaze[0])
                    frames.append(item["frameNumber"])
            
            # Image save and overlay
            if item["frameNumber"] < 2:
                img_data = base64.b64encode(item["thumbnail"]['binary'])
                with open("class.jpeg", "wb") as fh:
                    fh.write(base64.decodebytes(img_data))

    return positions_i

In [ ]:
def movementStatistics(positions_i):
  # movement average and standard deviation 
  all_mov_stdev = []
  all_mov_avg = []
  all_loc_dist = []
  agg_loc_dist = []

  positions_i = np.asarray(positions_i)
  positions_i = positions_i / 16
  standardized_pos = preprocessing.scale(positions_i)

  df = DataFrame(positions_i, columns=['x', 'y'])
  class_mov_stdev = statistics.stdev(df['x'])
  all_mov_stdev.append(class_mov_stdev)
  all_mov_avg.append([statistics.mean(df['x']), statistics.mean(df['y'])])

  positions_x = list(np.array(positions_i)[:,0]) 
  range_top = (np.percentile(positions_x, 95))
  range_bot = (np.percentile(positions_x, 5))
  full_range = range_top - range_bot
  ind_dist = []

  left_freq = (sum(i < (range_bot + full_range/3) for i in positions_x))    
  right_freq = (sum(i > (range_bot + 2*full_range/3) for i in positions_x))
  mid_freq = len(positions_x) - left_freq - right_freq
  ind_dist.append(int(left_freq))
  ind_dist.append(int(mid_freq))
  ind_dist.append(int(right_freq))

  all_loc_dist.append(ind_dist)

  all_loc_dist = list(all_loc_dist)
  for i in range(3):
    agg_loc_dist.append(int(sum(np.array(all_loc_dist)[:,i])))

  return agg_loc_dist

In [ ]:
def viewLocationCluster(im_path, positions_i):
  fig, ax = plt.subplots()
  im = Image.open(im_path)
  implot = plt.imshow(im)

  positions_i = np.asarray(positions_i)
  positions_i = positions_i / 16
  standardized_pos = preprocessing.scale(positions_i)

  df = DataFrame(positions_i, columns=['x', 'y'])
  df_std = DataFrame(standardized_pos, columns=['x', 'y'])

  plt.scatter(df['x'], df['y'])
  plt.axis("off")
  plt.show()

def viewLocationHeatmap(df, im_path):
  w, h = im.size
  hm, xe, ye = np.histogram2d(df['x'], df['y'], bins=[np.arange(0, w, 10), np.arange(0, h, 10)])
  extent = [0, 250, 0, 160]

  img1 = plt.imshow(im)
  img2 = plt.imshow(hm.T, extent=[0, w, h, 0], alpha=0.4, cmap="Reds")
  plt.axis("off")
  plt.savefig(im_path)
  plt.show()

Identify principal movement direction

In [ ]:
def computePrincipalComponent(df):
  X = np.array([df['x'], df['y']]).T
  pca = PCA(n_components=2)
  pca.fit(X)
  return pca 

def pcaVector(v_0, v_1):
    ax = plt.gca()
    ax.annotate('', v_1, v_0, arrowprops=dict(arrowstyle='->',linewidth=2,shrinkA=0, shrinkB=0))
    
def visualizePCA(im_path, pca):
  fig, ax = plt.subplots()
  ax.grid(False)
  ax.set_xlim([50, 350])
  im = Image.open(im_path)
  implot = plt.imshow(im)

  plt.scatter(X[:, 0], X[:, 1], alpha=0.6, c='r')
  for length, vector in zip(pca.explained_variance_, pca.components_):
      v = vector * 3 * np.sqrt(length)
      draw_vector(pca.mean_, pca.mean_ + v)

  plt.axis('equal');

def flattenX(pca):
  pc1_start = pca.mean_
  pc1_end = pca.mean_ + pca.components_[0]

  proj_X = []    
  for x in X:
      point = Point(x[0], x[1])
      line = LineString([(pc1_start[0], pc1_start[1]), (pc1_end[0], pc1_end[1])])

      x = np.array(point.coords[0])
      u = np.array(line.coords[0])
      v = np.array(line.coords[len(line.coords)-1])
      n = v - u
      n /= np.linalg.norm(n, 2)

      P = u + n*np.dot(x - u, n)
      proj_X.append(list(P))
  
  return proj_X

#### Student location inference

In [ ]:
def getStuLocations(collection, subject = "student", return_errors = False):
  maxStusFound = 0
  stuTracker = {} 

  for item in collection:
      positions_s = []
      stuID = []
      if item["channel"]=="student" and item["frameNumber"] < 100:
          
          for person in item["people"]:
              s_gaze = person["inference"]["head"]["gazeVector"]
              if len(s_gaze) == 2:
                  positions_s.append(s_gaze[0])
                  stuID.append(person["openposeId"])
              
          positions_s = np.asarray(positions_s)
          positions_s = positions_s / 16
          fig, ax = plt.subplots()
          
          # Image save and overlay
          img_data = base64.b64encode(item["thumbnail"]['binary'])
          with open(str(item["frameNumber"]) + ".jpeg", "wb") as fh:
              fh.write(base64.decodebytes(img_data))
          
          im = Image.open(str(item["frameNumber"]) + ".jpeg")
          
          # Student scatter plot
          df = DataFrame(positions_s, columns=['x', 'y'])
          kmeans = KMeans(n_clusters=3).fit(df)
          centroids = kmeans.cluster_centers_

          plt.scatter(df['x'], df['y'], c=kmeans.labels_, s=50, alpha=0.6)
          xx, yy = np.meshgrid(np.arange(0, 240, 1), 
                              np.arange(0, 134, 1))
          Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
          Z = Z.reshape(xx.shape)
          plt.contour(xx, yy, Z, colors='black')
      
          
          plt.show()

In [ ]:
# code modified from original program 
# source: https://www.pyimagesearch.com/2018/07/23/simple-object-tracking-with-opencv/

class CentroidTracker():
	def __init__(self, maxDisappeared=30):
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()
		self.poses = OrderedDict()
		self.maxDisappeared = maxDisappeared

	def register(self, centroid, pose):
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.poses[self.nextObjectID] = pose
		self.nextObjectID += 1

	def deregister(self, objectID):
		del self.objects[objectID]
		del self.disappeared[objectID]
		del self.poses[objectID]

	def update(self, rects, person_poses):
		if len(rects) == 0:
			for objectID in self.disappeared.keys():
				self.disappeared[objectID] += 1

				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)


			return self.objects, self.poses

		inputCentroids = np.zeros((len(rects), 2), dtype="int")

		# loop over bounding box rects
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)

		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i],person_poses[i])

		else:
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())

			D = dist.cdist(np.array(objectCentroids), inputCentroids)
			rows = D.min(axis=1).argsort()
			cols = D.argmin(axis=1)[rows]
			usedRows = set()
			usedCols = set()

			for (row, col) in zip(rows, cols):
				if row in usedRows or col in usedCols:
					continue

				objectID = objectIDs[row]
				dist_thresh = math.sqrt( (self.objects[objectID][0] - inputCentroids[col][0])**2 + (self.objects[objectID][1] - inputCentroids[col][1])**2 )
				if dist_thresh < 300:
					self.objects[objectID] = inputCentroids[col]
					self.poses[objectID] = person_poses[col]
				self.disappeared[objectID] = 0

				usedRows.add(row)
				usedCols.add(col)

			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			if D.shape[0] >= D.shape[1]:
				for row in unusedRows:
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)
			else:
				for col in unusedCols:
					self.register(inputCentroids[col],person_poses[col])

		return (self.objects, self.poses)